In [15]:
import torch
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch import nn

import torch.nn.functional as F
import torch.optim as optim


In [3]:
# Download training data from open datasets.
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 170M/170M [00:03<00:00, 49.3MB/s]


In [4]:
#Personal note. The set contains a set of image and label. Access image is data[i][0] and access label is data[i][1]
print(training_data.classes[training_data[0][1]] )


frog


In [5]:
batch_size = 64

#Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32])
Shape of y: torch.Size([64]) torch.int64


In [7]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

#Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1) # flatten all dimensions except batch
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [28]:
def train(dataloader, model , loss_fn, optimizer):
  for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
      #get the inputs which is a pair of inut, label
      inputs, labels = data
      inputs = inputs.to(device)
      labels = labels.to(device)

      #zero the parameter gradients
      optimizer.zero_grad()

      #forward + backward + optimize
      outputs = model(inputs)
      loss = loss_fn(outputs, labels)
      loss.backward()
      optimizer.step()

      #print statistics
      running_loss += loss.item()
      if i % 100 == 99:    # print every 2000 mini-batches
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
        running_loss = 0.0


In [26]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [31]:
num_epochs = 5
for epoch in range(num_epochs):
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

[1,   100] loss: 0.077
[1,   200] loss: 0.077
[1,   300] loss: 0.077
[1,   400] loss: 0.075
[1,   500] loss: 0.076
[1,   600] loss: 0.077
[1,   700] loss: 0.076
[2,   100] loss: 0.075
[2,   200] loss: 0.075
[2,   300] loss: 0.075
[2,   400] loss: 0.073
[2,   500] loss: 0.074
[2,   600] loss: 0.075
[2,   700] loss: 0.075
Test Error: 
 Accuracy: 46.9%, Avg loss: 1.476040 

[1,   100] loss: 0.073
[1,   200] loss: 0.073
[1,   300] loss: 0.073
[1,   400] loss: 0.072
[1,   500] loss: 0.072
[1,   600] loss: 0.074
[1,   700] loss: 0.073
[2,   100] loss: 0.072
[2,   200] loss: 0.071
[2,   300] loss: 0.072
[2,   400] loss: 0.070
[2,   500] loss: 0.071
[2,   600] loss: 0.072
[2,   700] loss: 0.071
Test Error: 
 Accuracy: 48.8%, Avg loss: 1.418166 

[1,   100] loss: 0.070
[1,   200] loss: 0.070
[1,   300] loss: 0.070
[1,   400] loss: 0.069
[1,   500] loss: 0.069
[1,   600] loss: 0.071
[1,   700] loss: 0.070
[2,   100] loss: 0.069
[2,   200] loss: 0.069
[2,   300] loss: 0.069
[2,   400] loss: 0.068

In [44]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

img = Image.open("/content/data/frog-610x343.jpg").convert("RGB")
img = img = img.resize((32, 32))
img_tensor = ToTensor()(img)
img_tensor = img_tensor.unsqueeze(0)

In [49]:
img_tensor = img_tensor.to(device)

In [55]:
model.eval()
with torch.no_grad():
    output = model(img_tensor)
    predicted_class = output.argmax(1).item()
print(f"Predicted class index: {training_data.classes[predicted_class]}")

Predicted class index: deer
